In [11]:
from keras.datasets import mnist # pip install keras tensorflow
import matplotlib.pyplot as plt # pip install matplotlib
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, GaussianDropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

# Вариант задания: Распознавание цифр на изображении (MNIST digits classification dataset)
# Нейросеть должна состоять из четырех полносвязных слоёв,
# обязательное использование GaussianDropout,
# в качестве оптимизатора использовать SGD

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train = X_train / 255.0
X_test = X_test / 255.0
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0], -1)
print(X_train.shape)
X_test = X_test.reshape(X_test.shape[0], -1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

(60000, 28, 28)
(60000, 784)


In [16]:
# https://www.kaggle.com/code/schmoyote/guide-to-mnist-digit-classification-with-keras
# версия самих разработчиков mnist
def mnist_version_model_builder():
    model = Sequential()

    model.add(Input(shape=(784,)))
    model.add(Dense(units=128, activation="relu")) # ReLU -> σ(x) = max(0, x)
    model.add(Dense(units=128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(units=10, activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# моя версия под собственный вариант задания
def my_version_model_builder():
    model = Sequential()

    model.add(Input(shape=(784,)))
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.5)) # GaussianDropout = Dropout(0.5)
    model.add(Dense(units=256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation="softmax"))  # Выходной слой

    sgd = SGD(learning_rate = 0.01, momentum = 0.9, nesterov = True)
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
    return model

#model = mnist_version_model_builder()
model = my_version_model_builder()
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 567,434 (2.16 MB)

 Trainable params: 567,434 (2.16 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
print(model.layers)
for W in model.get_weights(): print(W.shape, W.dtype)

[<Dense name=dense, built=True>, <Dropout name=dropout, built=True>, <Dense name=dense_1, built=True>, <Dropout name=dropout_1, built=True>, <Dense name=dense_2, built=True>, <Dropout name=dropout_2, built=True>, <Dense name=dense_3, built=True>]
(784, 512) float32
(512,) float32
(512, 256) float32
(256,) float32
(256, 128) float32
(128,) float32
(128, 10) float32
(10,) float32


In [6]:
early_stopping = EarlyStopping(monitor='accuracy', patience=float("inf"), verbose=1, mode='max', baseline=0.9999, restore_best_weights=True)
# - monitor: Метрическая величина, которую вы хотите отслеживать. В данном случае вы можете использовать `accuracy`.
# - patience: Количество эпох, которое нужно ждать, если метрика не улучшается, прежде чем остановить обучение. Если вы установите его в 0, обучение остановится сразу при достижении точности 0.9999.
# - verbose: Уровень отображения информации о процессе остановки. Установив его в 1, вы получите текстовый вывод. 
# - mode: Режим отслеживания. 'max' указывает, что нужно остановить обучение, если точность достигает максимума.
# - baseline: Начальное значение для метрики. В данном случае мы устанавливаем его в 0.9999, чтобы остановить обучение, если метрика достигает этого значения.
# - restore_best_weights: Если `True`, то веса модели будут восстановлены на уровне, который соответствует лучшему значению `monitor`.

In [8]:
BATCH_SIZE = 512
epochs = 11
while True:
    history = model.fit(x=X_train, y=y_train, batch_size = BATCH_SIZE, epochs = epochs, verbose=1, callbacks=[early_stopping])
    acc = max(history.history["accuracy"])
    if acc > 0.99: train_loss, train_acc = model.evaluate(X_train, y_train, verbose=1)
    else: train_loss = train_acc = -1 
    print(history.history, acc, train_loss, train_acc)
    if acc > 0.9999 or train_acc > 0.9999: break

Epoch 1/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9973 - loss: 0.0096
Epoch 2/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9970 - loss: 0.0103
Epoch 3/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9973 - loss: 0.0097
Epoch 4/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9972 - loss: 0.0093
Epoch 5/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9967 - loss: 0.0111
Epoch 6/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9964 - loss: 0.0110
Epoch 7/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9965 - loss: 0.0101
Epoch 8/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9970 - loss: 0.0101
Epoch 9/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9969 - loss: 0.0108
Epoch 10/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9974 - loss: 0.0090
Epoch 11/11
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9969 - loss: 0.0106
Restoring model weights from the end of the best epo

In [37]:
np.set_printoptions(formatter={"float_kind": "{:.4f}".format})
#image = np.random.rand(28, 28).astype(np.float32)
#print(tuple(tuple(round(x * 255) for x in row) for row in image))
image = np.array(((41, 148, 2, 80, 128, 191, 188, 115, 125, 96, 16, 128, 157, 10, 227, 44, 183, 197, 59, 249, 254, 39, 116, 58, 118, 120, 51, 78), (78, 188, 163, 202, 213, 116, 80, 122, 75, 235, 116, 18, 48, 31, 143, 207, 223, 92, 57, 87, 12, 181, 202, 82, 143, 119, 69, 38), (83, 113, 251, 229, 117, 228, 240, 119, 223, 200, 91, 197, 97, 246, 169, 67, 131, 202, 127, 199, 122, 63, 209, 46, 57, 181, 65, 21), (127, 81, 49, 87, 99, 135, 34, 75, 53, 225, 39, 108, 249, 171, 75, 34, 194, 210, 113, 2, 67, 72, 15, 188, 250, 65, 196, 114), (94, 185, 75, 233, 35, 156, 240, 94, 133, 77, 53, 134, 238, 180, 215, 110, 141, 70, 217, 84, 64, 35, 244, 186, 33, 137, 129, 217), (195, 70, 230, 92, 99, 111, 53, 237, 248, 25, 233, 224, 87, 32, 38, 90, 57, 161, 46, 53, 133, 9, 183, 223, 205, 93, 83, 81), (44, 241, 76, 85, 142, 134, 44, 239, 93, 24, 99, 171, 237, 52, 147, 89, 64, 41, 233, 194, 66, 10, 44, 143, 90, 79, 219, 118), (19, 202, 50, 226, 214, 203, 95, 121, 144, 4, 196, 3, 178, 163, 81, 208, 35, 179, 25, 100, 98, 247, 91, 146, 20, 205, 204, 186), (250, 213, 25, 8, 139, 63, 111, 137, 95, 82, 144, 24, 181, 194, 29, 116, 216, 121, 239, 187, 38, 78, 202, 185, 50, 199, 93, 146), (81, 7, 53, 236, 121, 165, 151, 85, 224, 120, 4, 100, 161, 148, 49, 227, 212, 102, 103, 58, 32, 82, 163, 91, 218, 151, 77, 125), (128, 23, 100, 175, 97, 27, 197, 99, 67, 26, 128, 128, 76, 41, 162, 120, 212, 107, 126, 3, 14, 228, 65, 142, 232, 181, 1, 13), (56, 51, 48, 89, 179, 37, 93, 143, 85, 80, 186, 132, 243, 89, 237, 6, 105, 193, 232, 175, 98, 162, 12, 79, 59, 227, 39, 149), (186, 164, 150, 6, 23, 246, 232, 174, 173, 29, 191, 98, 93, 179, 63, 77, 154, 53, 63, 178, 13, 237, 209, 146, 195, 177, 25, 80), (166, 56, 184, 16, 45, 117, 18, 135, 61, 114, 104, 83, 105, 52, 47, 192, 32, 20, 82, 215, 202, 71, 27, 161, 91, 97, 183, 130), (83, 240, 194, 9, 93, 67, 98, 33, 133, 86, 238, 238, 86, 118, 157, 101, 19, 172, 33, 18, 212, 79, 36, 59, 159, 128, 145, 232), (96, 160, 212, 8, 186, 194, 80, 36, 166, 55, 233, 113, 160, 163, 95, 5, 231, 176, 191, 99, 114, 82, 75, 205, 116, 163, 169, 55), (204, 78, 66, 108, 17, 44, 157, 221, 73, 31, 45, 223, 166, 186, 71, 195, 96, 148, 19, 94, 88, 223, 72, 156, 47, 232, 49, 180), (83, 69, 87, 26, 95, 233, 120, 165, 224, 135, 187, 28, 165, 68, 251, 85, 220, 0, 177, 19, 100, 101, 122, 230, 204, 222, 124, 178), (146, 114, 66, 213, 48, 50, 171, 53, 196, 130, 44, 156, 179, 80, 3, 209, 127, 76, 119, 6, 240, 103, 167, 69, 192, 69, 121, 240), (6, 217, 131, 111, 207, 65, 1, 247, 202, 132, 119, 74, 11, 243, 72, 169, 224, 144, 45, 218, 93, 253, 161, 132, 191, 73, 176, 126), (116, 18, 149, 78, 138, 127, 193, 128, 253, 196, 222, 251, 94, 78, 166, 189, 68, 167, 67, 9, 57, 225, 106, 66, 42, 153, 182, 8), (149, 94, 116, 111, 1, 243, 148, 167, 200, 203, 92, 53, 135, 73, 168, 64, 10, 48, 159, 4, 230, 142, 245, 69, 247, 156, 241, 45), (55, 240, 87, 240, 68, 215, 121, 152, 62, 49, 21, 93, 137, 187, 196, 70, 153, 101, 69, 70, 111, 78, 126, 43, 85, 226, 214, 252), (101, 254, 41, 109, 222, 80, 54, 145, 20, 57, 97, 152, 25, 4, 13, 246, 177, 3, 161, 1, 145, 243, 63, 84, 244, 187, 218, 4), (154, 139, 81, 14, 24, 107, 44, 94, 30, 19, 192, 56, 202, 4, 208, 17, 32, 121, 177, 166, 36, 36, 38, 41, 27, 165, 24, 39), (156, 5, 191, 24, 240, 63, 21, 18, 70, 34, 179, 245, 156, 168, 247, 87, 196, 186, 18, 142, 89, 75, 91, 80, 217, 86, 55, 14), (162, 1, 224, 82, 40, 196, 33, 2, 187, 227, 151, 205, 64, 130, 97, 242, 62, 133, 29, 23, 145, 116, 28, 162, 235, 92, 105, 104), (192, 37, 68, 116, 107, 11, 191, 239, 198, 225, 113, 188, 106, 206, 147, 97, 97, 107, 92, 200, 0, 55, 52, 119, 40, 39, 22, 233)))
image = image / 255

#image = np.expand_dims(image, axis=2) # axis=0 => (1, 28, 28) | axis=1 => (28, 1, 28) | axis=2 => (28, 28, 1)
image5 = image.reshape(28 * 28)
image5 = np.array((image5,) * 5)
image = image.reshape(1, 28 * 28)
print(image.shape)
print(image5.shape)

res = model.predict(image5)
print(res)
#print(dir(model))
print(model.layers[0].input)
print(model.layers[0].output)

from keras.models import Model
activation_model = Model(inputs=model.layers[0].input, outputs=[layer.output for layer in model.layers])

# Получение активаций для входного изображения
activations = activation_model.predict(image)

for i, activation in enumerate(activations):
    print(f"Activation of layer {i}: shape {activation.shape}")

for i, layer in enumerate(model.layers):
    w = layer.get_weights()
    if not w: continue # dropout'ы не содержат весов
    weights, biases = w
    print(f"Weights of layer {i}: shape {weights.shape}, biases shape {biases.shape}")

(1, 784)
(5, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.0000 0.0000 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.9998 0.0000]
 [0.0000 0.0000 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.9998 0.0000]
 [0.0000 0.0000 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.9998 0.0000]
 [0.0000 0.0000 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.9998 0.0000]
 [0.0000 0.0000 0.0001 0.0001 0.0000 0.0000 0.0000 0.0000 0.9998 0.0000]]
<KerasTensor shape=(None, 784), dtype=float32, sparse=False, name=input_layer>
<KerasTensor shape=(None, 512), dtype=float32, sparse=False, name=keras_tensor_174>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Activation of layer 0: shape (1, 512)
Activation of layer 1: shape (1, 512)
Activation of layer 2: shape (1, 256)
Activation of layer 3: shape (1, 256)
Activation of layer 4: shape (1, 128)
Activation of layer 5: shape (1, 128)
Activation of layer 6: shape (1, 10)
Weights of layer 0: shape (784, 512), biases shape (512,)
Weights of layer 2: shape (512, 256), biases shape (25

In [38]:
from keras.models import load_model

option = 3

if option == 0:
    model.save("trained_model.keras")
    print("saved #1")
if option == 1:
    model.save("trained_model2.keras")
    print("saved #2")

if option in (2, 3):
    print("load model...")
    if option == 2: model = load_model("trained_model.keras")
    if option == 3: model = load_model("trained_model2.keras")

load model...
